In [1]:
import pandas as pd
import mne as mne
import os 
import time
import numpy as np
import matplotlib.pyplot as plt
import joblib
import constants
from IPython.utils import io
import time
import sys
import yasa
from scipy.signal import welch

from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier

from sklearn.metrics import accuracy_score

#Import my modules
import format_eeg_data
import constants
import eeg_stat_ts
import run_expts

#TS Fresh Parameter Settings
from tsfresh.feature_extraction import ComprehensiveFCParameters
from tsfresh.feature_extraction import MinimalFCParameters
from tsfresh.feature_extraction import EfficientFCParameters
from tsfresh.feature_extraction import extract_features

# Set display options to show all rows and columns
pd.set_option('display.max_rows', 50)  # Show rows
pd.set_option('display.max_columns', 160)  # Show columns

/user/home/ko20929/.conda/envs/sktime_latest/lib/python3.11/site-packages/antropy/fractal.py:197: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit((types.Array(types.float64, 1, "C", readonly=True), types.int32))


In [2]:
%%capture captured_output
#generate all path names and class list(s) etc. 
channels = constants.channel_list
paths , class_list, sleep_night_list , sleep_type_list , participant_id_list = constants.generate_paths_and_info(blue_pebble = True)

In [7]:
folder = '/user/home/ko20929/work/RBD_using_custom_package/Blue_pebble/'
data_type = ['Wake', 'N1', 'N2','N3', 'REM'][0]
for data_type in ['Wake', 'N1', 'N2','N3', 'REM']:
    paths = joblib.load(folder + data_type + '_paths.pkl')
    num_paths = ' : ' + str( len(paths['selected_paths']) )
    print(data_type + num_paths)

Wake : 56
N1 : 71
N2 : 71
N3 : 72
REM : 69


In [5]:
%%capture captured_output
data_epo = mne.read_epochs(paths['selected_paths'][0])
df_full = data_epo.to_data_frame()

In [6]:
df_full

,time,condition,epoch,Fp1,Fp2,F3,F4,C3,C4,P3,P4,O1,O2,F7,F8,T7,T8,P7,P8,Fz,Cz,Pz,Oz,FT9,FT10,POz,Fpz,AF7,AF8,F1,F2,F5,F6,FC1,FC2,FC3,FC4,FC5,FC6,FT7,FT8,C1,C2,C5,C6,CP1,CP2,CP3,CP4,CP5,CP6,TP7,TP8,P1,P2,P5,P6,PO7,PO8,FCz
0,0.000000,REM,300,-2.032487,-2.680647,2.925733,0.807765,4.288162,5.719256,3.577327,-2.542708,-2.950570,-2.613520,-4.767934,-2.604933,-6.580829,5.129007,-4.827203,-1.752456,-1.701498,5.982833,4.242734,-1.838870,-6.457373,-0.991300,-0.209091,-1.189364,-0.937875,-3.808327,3.075270,-0.614968,2.292314,-2.608716,10.705877,0.737028,7.349821,2.655336,-5.175828,2.494049,-3.463536,0.511765,3.938487,4.387168,-2.613156,4.946236,0.198124,0.571666,2.528995,-1.197500,-0.793985,2.692628,-5.409685,-6.304156,3.808599,-0.490593,1.385630,-3.153811,-2.660386,-6.300166,4.321662
1,0.003906,REM,300,-1.307057,-2.559225,5.593601,1.888171,5.504160,5.316182,2.955149,-4.739616,-5.518667,-6.395855,-3.769040,-2.652371,-7.453790,3.554353,-5.006923,-4.446731,-0.093299,6.740174,2.518042,-4.346412,-6.504402,-1.493606,-3.095440,-0.707676,0.071945,-3.216051,5.485228,1.350180,4.317089,-1.303275,13.426407,2.320674,9.841879,3.411124,-3.525391,2.819234,-2.328486,0.272545,5.549669,4.705355,-1.346179,4.073940,0.501091,0.095360,2.868287,-2.642008,-0.953990,1.260516,-5.284287,-7.220646,2.984648,-1.991499,1.283415,-5.338111,-3.200795,-8.706520,6.438930
2,0.007812,REM,300,-0.054787,-1.093277,7.572148,3.253002,5.463611,4.953353,1.337279,-6.283917,-7.584512,-8.270707,-2.167638,-1.664441,-6.743136,2.921844,-5.654505,-6.155975,2.107063,7.159654,0.431306,-6.891920,-5.514492,-1.436547,-5.782750,0.699416,1.144026,-1.789697,7.464241,3.474194,5.677988,0.564474,15.086477,4.052999,10.498465,4.442259,-2.607600,3.235285,-1.165735,0.407689,6.027453,5.016293,-0.792927,3.182762,-0.210364,-0.515606,1.775277,-3.944500,-1.902614,-0.311127,-5.231992,-7.917035,1.138436,-3.847707,0.122294,-6.905473,-4.295419,-10.640515,8.167624
3,0.011719,REM,300,1.257322,1.186034,8.188959,4.677579,4.392113,4.557769,-0.738576,-6.734385,-8.469859,-7.181682,-0.555312,-0.092031,-4.693518,3.438018,-6.573805,-6.212212,4.424140,7.235842,-1.104161,-8.582126,-4.253762,-1.003825,-7.580882,2.429774,1.940258,-0.166790,8.492288,5.083100,5.919408,2.435615,15.458149,5.285035,9.189599,5.104644,-2.634972,3.354974,-0.519535,0.758653,5.592642,5.287399,-1.185510,2.204859,-1.483416,-1.108735,-0.374960,-4.829283,-3.364503,-1.493618,-5.366478,-8.455116,-0.942365,-5.478361,-1.675863,-7.273326,-5.548665,-11.456358,9.239815
4,0.015625,REM,300,2.207770,3.126814,7.417061,5.782129,2.763926,3.956621,-2.470975,-6.331020,-8.462913,-4.285724,0.249668,1.059620,-2.962304,4.307563,-7.065192,-5.111169,5.878437,6.799359,-1.579038,-8.716666,-3.129551,-0.310069,-8.212351,3.699831,2.322165,1.065241,8.427510,5.604807,5.397720,3.731804,14.698342,5.619105,6.841960,4.905894,-3.016243,2.985653,-0.193900,1.199112,4.761877,5.256946,-1.849322,1.298743,-2.724913,-1.482881,-2.449068,-5.204415,-4.601509,-1.756327,-5.236480,-8.506535,-2.277366,-6.300543,-2.976200,-6.543101,-6.081513,-10.909582,9.381191
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
698875,29.980469,REM,410,-1.185658,-3.043654,10.955294,6.506017,4.960727,7.569882,-1.601313,-5.308775,-7.710665,-8.565495,2.657336,4.283674,-6.686386,-3.836473,-9.342273,-9.732209,10.085852,0.986207,-6.516456,-7.013576,-10.721165,-1.562455,-5.061646,-0.579206,5.066321,2.056700,8.167183,8.268870,5.467676,9.365591,11.821610,4.688401,7.027687,6.435895,1.477679,1.948162,-2.400164,1.274172,6.616472,3.290433,-0.872850,0.665445,-1.871130,0.773781,-0.796455,2.573838,-3.265256,0.247251,-8.139217,-2.984905,-1.890232,-7.806426,-3.110429,-5.147682,-7.613734,-8.006091,7.133820
698876,29.984375,REM,410,-0.224495,-2.473509,12.499704,5.461541,5.908885,8.007769,-1.648557,-5.881816,-7.315998,-8.864930,3.208985,3.629643,-6.76

#### Code to do TS Fresh Expts on BPW TS 

In [9]:
load_path = '/user/home/ko20929/work/RBD_using_custom_package/Data/freq_6_second_files/'
data_types = ['Wake','N1', 'N2', 'N3', 'REM']
loaded_data = {}

In [10]:
X = pd.read_hdf(load_path + data_type +  'six_second_freq_df.h5', key='df', mode='r')
y = pd.read_hdf(load_path + data_type +  '_y.h5', key='df', mode='r') 
groups = pd.read_hdf(load_path + data_type +  '_groups.h5', key='df', mode='r')
X, y , groups = X.reset_index(drop = True) , y.reset_index(drop = True) , groups.reset_index(drop = True)

In [13]:
X.head(3)

,Delta_Central,Theta_Central,Alpha_Central,Sigma_Central,Beta_Central,Gamma_Central,TotalAbsPow_Central,Delta_Frontal,Theta_Frontal,Alpha_Frontal,Sigma_Frontal,Beta_Frontal,Gamma_Frontal,TotalAbsPow_Frontal,Delta_Left Central,Theta_Left Central,Alpha_Left Central,Sigma_Left Central,Beta_Left Central,Gamma_Left Central,TotalAbsPow_Left Central,Delta_Left Frontal,Theta_Left Frontal,Alpha_Left Frontal,Sigma_Left Frontal,Beta_Left Frontal,Gamma_Left Frontal,TotalAbsPow_Left Frontal,Delta_Left Parietal,Theta_Left Parietal,Alpha_Left Parietal,Sigma_Left Parietal,Beta_Left Parietal,Gamma_Left Parietal,TotalAbsPow_Left Parietal,Delta_Left Temporal,Theta_Left Temporal,Alpha_Left Temporal,Sigma_Left Temporal,Beta_Left Temporal,Gamma_Left Temporal,TotalAbsPow_Left Temporal,Delta_Occipital,Theta_Occipital,Alpha_Occipital,Sigma_Occipital,Beta_Occipital,Gamma_Occipital,TotalAbsPow_Occipital,Delta_Parietal,Theta_Parietal,Alpha_Parietal,Sigma_Parietal,Beta_Parietal,Gamma_Parietal,TotalAbsPow_Parietal,Delta_Prefrontal,Theta_Prefrontal,Alpha_Prefrontal,Sigma_Prefrontal,Beta_Prefrontal,Gamma_Prefrontal,TotalAbsPow_Prefrontal,Delta_Right Central,Theta_Right Central,Alpha_Right Central,Sigma_Right Central,Beta_Right Central,Gamma_Right Central,TotalAbsPow_Right Central,Delta_Right Frontal,Theta_Right Frontal,Alpha_Right Frontal,Sigma_Right Frontal,Beta_Right Frontal,Gamma_Right Frontal,TotalAbsPow_Right Frontal,Delta_Right Parietal,Theta_Right Parietal,Alpha_Right Parietal,Sigma_Right Parietal,Beta_Right Parietal,Gamma_Right Parietal,TotalAbsPow_Right Parietal,Delta_Right Temporal,Theta_Right Temporal,Alpha_Right Temporal,Sigma_Right Temporal,Beta_Right Temporal,Gamma_Right Temporal,TotalAbsPow_Right Temporal
0,0 0.534534 1 0.387710 2 0.42522...,0 0.293270 1 0.424379 2 0.26447...,0 0.086945 1 0.067723 2 0.11175...,0 0.039545 1 0.053726 2 0.10141...,0 0.034052 1 0.051924 2 0.08084...,0 0.011654 1 0.014538 2 0.01627...,0 42.921243 1 44.265962 2 26.27...,0 0.562205 1 0.224364 2 0.28685...,0 0.234471 1 0.525861 2 0.33281...,0 0.124148 1 0.073444 2 0.16759...,0 0.028440 1 0.043820 2 0.13036...,0 0.029661 1 0.084813 2 0.06009...,0 0.021075 1 0.047698 2 0.02227...,0 54.022768 1 40.689773 2 31.86...,0 0.437704 1 0.346170 2 0.28530...,0 0.386208 1 0.252447 2 0.37986...,0 0.077953 1 0.119673 2 0.12402...,0 0.053551 1 0.132362 2 0.12090...,0 0.035913 1 0.125871 2 0.07847...,0 0.008672 1 0.023476 2 0.01142...,0 33.288644 1 20.460392 2 19.77...,0 0.507343 1 0.377686 2 0.43015...,0 0.282232 1 0.241919 2 0.22600...,0 0.105453 1 0.076300 2 0.09498...,0 0.050939 1 0.111277 2 0.15029...,0 0.042715 1 0.158444 2 0.08374...,0 0.011319 1 0.034374 2 0.01482...,0 36.446826 1 23.020309 2 26.20...,0 0.392568 1 0.343390 2 0.33597...,0 0.451588 1 0.373333 2 0.27910...,0 0.094293 1 0.091857 2 0.18340...,0 0.029777 1 0.079197 2 0.11679...,0 0.023061 1 0.084489 2 0.06996...,0 0.008713 1 0.027734 2 0.01475...,0 36.112410 1 22.464889 2 17.19...,0 0.483605 1 0.320194 2 0.38395...,0 0.425946 1 0.426219 2 0.33597...,0 0.044068 1 0.112796 2 0.11753...,0 0.017517 1 0.064903 2 0.09575...,0 0.022067 1 0.057587 2 0.05730...,0 0.006797 1 0.018302 2 0.00947...,0 46.189099 1 25.445620 2 26.65...,0 0.621432 1 0.428326 2 0.41906...,0 0.230893 1 0.380055 2 0.28392...,0 0.085350 1 0.072859 2 0.12308...,0 0.026619 1 0.028091 2 0.09972...,0 0.023332 1 0.060053 2 0.05760...,0 0.012374 1 0.030616 2 0.01659...,0 37.964128 1 26.262460 2 19.64...,0 0.510837 1 0.590653 2 0.51721...,0 0.345385 1 0.125599 2 0.21988...,0 0.091472 1 0.107587 2 0.09551...,0 0.019926 1 0.057454 2 0.08699...,0 0.024483 1 0.090324 2 0.06777...,0 0.007898 1 0.028383 2 0.01260...,0 32.353870 1 16.727743 2 21.38...,0 0.551702 1 0.520832 2 0.49894...,0 0.323926 1 0.192152 2 0.24464...,0 0.063311 1 0.096004 2 0.09295...,0 0.026042 1 0.071211 2 0.08661...,0 0.028445 1 0.097041 2 0.06663...,0 0.006574 1 0.022760 2 0.01020...,0 26.396182 1 17.208396 2 22.11...,0 0.486400 1 0.251682 2 0.29209...,0 0.320065 1 0.377744 2 0.31032...,0 0.0

In [14]:
#Transform the X into TS_Fresh Features___
#1. Convert to TS_Fresh format Dataframe 
ts_fresh_df = format_eeg_data.convert_sktime_df_to_ts_fresh_format(X, ts_cols = list(X.columns))

In [15]:
ts_fresh_df

,id,Delta_Central,Theta_Central,Alpha_Central,Sigma_Central,Beta_Central,Gamma_Central,TotalAbsPow_Central,Delta_Frontal,Theta_Frontal,Alpha_Frontal,Sigma_Frontal,Beta_Frontal,Gamma_Frontal,TotalAbsPow_Frontal,Delta_Left Central,Theta_Left Central,Alpha_Left Central,Sigma_Left Central,Beta_Left Central,Gamma_Left Central,TotalAbsPow_Left Central,Delta_Left Frontal,Theta_Left Frontal,Alpha_Left Frontal,Sigma_Left Frontal,Beta_Left Frontal,Gamma_Left Frontal,TotalAbsPow_Left Frontal,Delta_Left Parietal,Theta_Left Parietal,Alpha_Left Parietal,Sigma_Left Parietal,Beta_Left Parietal,Gamma_Left Parietal,TotalAbsPow_Left Parietal,Delta_Left Temporal,Theta_Left Temporal,Alpha_Left Temporal,Sigma_Left Temporal,Beta_Left Temporal,Gamma_Left Temporal,TotalAbsPow_Left Temporal,Delta_Occipital,Theta_Occipital,Alpha_Occipital,Sigma_Occipital,Beta_Occipital,Gamma_Occipital,TotalAbsPow_Occipital,Delta_Parietal,Theta_Parietal,Alpha_Parietal,Sigma_Parietal,Beta_Parietal,Gamma_Parietal,TotalAbsPow_Parietal,Delta_Prefrontal,Theta_Prefrontal,Alpha_Prefrontal,Sigma_Prefrontal,Beta_Prefrontal,Gamma_Prefrontal,TotalAbsPow_Prefrontal,Delta_Right Central,Theta_Right Central,Alpha_Right Central,Sigma_Right Central,Beta_Right Central,Gamma_Right Central,TotalAbsPow_Right Central,Delta_Right Frontal,Theta_Right Frontal,Alpha_Right Frontal,Sigma_Right Frontal,Beta_Right Frontal,Gamma_Right Frontal,TotalAbsPow_Right Frontal,Delta_Right Parietal,Theta_Right Parietal,Alpha_Right Parietal,Sigma_Right Parietal,Beta_Right Parietal,Gamma_Right Parietal,TotalAbsPow_Right Parietal,Delta_Right Temporal,Theta_Right Temporal,Alpha_Right Temporal,Sigma_Right Temporal,Beta_Right Temporal,Gamma_Right Temporal,TotalAbsPow_Right Temporal,time
0,0,0.534534,0.293270,0.086945,0.039545,0.034052,0.011654,42.921243,0.562205,0.234471,0.124148,0.028440,0.029661,0.021075,54.022768,0.437704,0.386208,0.077953,0.053551,0.035913,0.008672,33.288644,0.507343,0.282232,0.105453,0.050939,0.042715,0.011319,36.446826,0.392568,0.451588,0.094293,0.029777,0.023061,0.008713,36.112410,0.483605,0.425946,0.044068,0.017517,0.022067,0.006797,46.189099,0.621432,0.230893,0.085350,0.026619,0.023332,0.012374,37.964128,0.510837,0.345385,0.091472,0.019926,0.024483,0.007898,32.353870,0.551702,0.323926,0.063311,0.026042,0.028445,0.006574,26.396182,0.486400,0.320065,0.096851,0.048139,0.038941,0.009603,28.684843,0.519843,0.240482,0.122410,0.060703,0.043114,0.013449,31.172679,0.512691,0.307831,0.113994,0.027299,0.027003,0.011182,37.848887,0.582064,0.245039,0.099080,0.037039,0.028650,0.008128,31.929320,0
1,0,0.387710,0.424379,0.067723,0.053726,0.051924,0.014538,44.265962,0.224364,0.525861,0.073444,0.043820,0.084813,0.047698,40.689773,0.346170,0.252447,0.119673,0.132362,0.125871,0.023476,20.460392,0.377686,0.241919,0.076300,0.111277,0.158444,0.034374,23.020309,0.343390,0.373333,0.091857,0.079197,0.084489,0.027734,22.464889,0.320194,0.426219,0.112796,0.064903,0.057587,0.018302,25.445620,0.428326,0.380055,0.072859,0.028091,0.060053,0.030616,26.262460,0.590653,0.125599,0.107587,0.057454,0.090324,0.028383,16.727743,0.520832,0.192152,0.096004,0.071211,0.097041,0.022760,17.208396,0.251682,0.377744,0.111577,0.137229,0.104430,0.017339,20.192532,0.442570,0.224037,0.080635,0.090135,0.130017,0.032606,22.419908,0.343574,0.389943,0.122130,0.042012,0.075496,0.026845,25.030353,0.477176,0.323402,0.070844,0.052024,0.064638,0.011917,25.144897,1
2,0,0.425226,0.264477,0.111759,0.101412,0.080848,0.016277,26.271356,0.286857,0.332811,0.167598,0.130364,0.060095,0.022276,31.867403,0.285308,0.379861,0.124025,0.120908,0.078475,0.011423,19.777598,0.430156,0.226001,0.094985,0.150294,0.083743,0.014822,26.200308,0.335971,0.279108,0.183408,0.116791,0.069968,0.014754,17.197335,0.383951,0.335979,0.117538,0.095755,0.057305,0.009472,26.650545,0.419065,0.283925,0.123085,0.099720,0.057609,0.016596,19.644615,0.517214,0.219884,0.095516,0.086999,0.067777,0.012609,21.384752,0.498946,0.244644,0.092959,0.086614,0.066636,0.010201,22.119606,0.292092,0.310328,

In [17]:
#2. Extract TS_Fresh Features from the dataframe
t1 = time.time()
settings = MinimalFCParameters()
extracted_ts_fresh_df = extract_features(ts_fresh_df, column_id = 'id' , column_sort = 'time',  default_fc_parameters=settings)
t2 = time.time()
t2-t1

13-Dec-23 15:22:42 | WARNING | Dependency not available for matrix_profile, this feature will be disabled!
Feature Extraction: 100%|██████████| 119/119 [00:01<00:00, 96.08it/s] 


2.027172088623047

In [18]:
# 3. Asign extract_ts_fresh_df to the variable X
X = extracted_ts_fresh_df.copy()
#Drop columns where all values are NA 
X = X.dropna(axis = 1)

#Drop columns where all values are the same
# Find columns where all values are the same
same_value_columns = X.columns[X.nunique() == 1]
# Drop columns with the same values
X = X.drop(columns=same_value_columns)

In [19]:
X

,Theta_Left Central__sum_values,Theta_Left Central__median,Theta_Left Central__mean,Theta_Left Central__length,Theta_Left Central__standard_deviation,Theta_Left Central__variance,Theta_Left Central__root_mean_square,Theta_Left Central__maximum,Theta_Left Central__absolute_maximum,Theta_Left Central__minimum,Alpha_Left Central__sum_values,Alpha_Left Central__median,Alpha_Left Central__mean,Alpha_Left Central__length,Alpha_Left Central__standard_deviation,Alpha_Left Central__variance,Alpha_Left Central__root_mean_square,Alpha_Left Central__maximum,Alpha_Left Central__absolute_maximum,Alpha_Left Central__minimum,Sigma_Left Central__sum_values,Sigma_Left Central__median,Sigma_Left Central__mean,Sigma_Left Central__length,Sigma_Left Central__standard_deviation,Sigma_Left Central__variance,Sigma_Left Central__root_mean_square,Sigma_Left Central__maximum,Sigma_Left Central__absolute_maximum,Sigma_Left Central__minimum,Beta_Left Central__sum_values,Beta_Left Central__median,Beta_Left Central__mean,Beta_Left Central__length,Beta_Left Central__standard_deviation,Beta_Left Central__variance,Beta_Left Central__root_mean_square,Beta_Left Central__maximum,Beta_Left Central__absolute_maximum,Beta_Left Central__minimum,Gamma_Left Central__sum_values,Gamma_Left Central__median,Gamma_Left Central__mean,Gamma_Left Central__length,Gamma_Left Central__standard_deviation,Gamma_Left Central__variance,Gamma_Left Central__root_mean_square,Gamma_Left Central__maximum,Gamma_Left Central__absolute_maximum,Gamma_Left Central__minimum,TotalAbsPow_Left Central__sum_values,TotalAbsPow_Left Central__median,TotalAbsPow_Left Central__mean,TotalAbsPow_Left Central__length,TotalAbsPow_Left Central__standard_deviation,TotalAbsPow_Left Central__variance,TotalAbsPow_Left Central__root_mean_square,TotalAbsPow_Left Central__maximum,TotalAbsPow_Left Central__absolute_maximum,TotalAbsPow_Left Central__minimum,Delta_Left Frontal__sum_values,Delta_Left Frontal__median,Delta_Left Frontal__mean,Delta_Left Frontal__length,Delta_Left Frontal__standard_deviation,Delta_Left Frontal__variance,Delta_Left Frontal__root_mean_square,Delta_Left Frontal__maximum,Delta_Left Frontal__absolute_maximum,Delta_Left Frontal__minimum,Theta_Left Frontal__sum_values,Theta_Left Frontal__median,Theta_Left Frontal__mean,Theta_Left Frontal__length,Theta_Left Frontal__standard_deviation,Theta_Left Frontal__variance,Theta_Left Frontal__root_mean_square,Theta_Left Frontal__maximum,Theta_Left Frontal__absolute_maximum,Theta_Left Frontal__minimum,...,TotalAbsPow_Right Parietal__sum_values,TotalAbsPow_Right Parietal__median,TotalAbsPow_Right Parietal__mean,TotalAbsPow_Right Parietal__length,TotalAbsPow_Right Parietal__standard_deviation,TotalAbsPow_Right Parietal__variance,TotalAbsPow_Right Parietal__root_mean_square,TotalAbsPow_Right Parietal__maximum,TotalAbsPow_Right Parietal__absolute_maximum,TotalAbsPow_Right Parietal__minimum,Delta_Right Temporal__sum_values,Delta_Right Temporal__median,Delta_Right Temporal__mean,Delta_Right Temporal__length,Delta_Right Temporal__standard_deviation,Delta_Right Temporal__variance,Delta_Right Temporal__root_mean_square,Delta_Right Temporal__maximum,Delta_Right Temporal__absolute_maximum,Delta_Right Temporal__minimum,Theta_Right Temporal__sum_values,Theta_Right Temporal__median,Theta_Right Temporal__mean,Theta_Right Temporal__length,Theta_Right Temporal__standard_deviation,Theta_Right Temporal__variance,Theta_Right Temporal__root_mean_square,Theta_Right Temporal__maximum,Theta_Right Temporal__absolute_maximum,Theta_Right Temporal__minimum,Alpha_Right Temporal__sum_values,Alpha_Right Temporal__median,Alpha_Right Temporal__mean,Alpha_Right Temporal__length,Alpha_Right Temporal__standard_deviation,Alpha_Right Temporal__variance,Alpha_Right Temporal__root_mean_square,Alpha_Right Temporal__maximum,Alpha_Right Temporal__absolute_maximum,Alpha_Right Temporal__minimum,Sigma_Right Temporal__sum_values,Sigma_Right Temporal__median,Sigma_Right Temporal__mean,Sigma_Right Temporal